In [8]:
#test the ollama local setup

from llama_index.llms.ollama import Ollama

llm = Ollama(model="phi3", request_timeout=60.0)

response = llm.complete("What color is Guinness beer?")
print(response)

Guinness beer, the iconic Irish stout, is typically a dark brown or black in color. This deep hue comes from its high concentration of melanoidins, which are created during the brewing process when malted barley and roasted unmalted barley (called "black patent") are used in the beer's recipe.

However, it is worth noting that there may also be a lighter variety of Guinness called "Guinness Foreign Extra Stout" which has a slightly different color and appearance compared to the traditional dark stout. This variation uses roasted barley in its brewing process but at a lower level than the mainstream version, resulting in a less intense black-brown hue.


In [12]:
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import Settings

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="microsoft/Phi-3-mini-4k-instruct-gguf"
# )

In [15]:
#test olamma embedding

from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="phi3",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

pass_embedding = ollama_embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], show_progress=True
)
print(pass_embedding)

query_embedding = ollama_embedding.get_query_embedding("Where is blue?")
print(query_embedding)

Generating embeddings: 100%|██████████| 2/2 [00:04<00:00,  2.28s/it]


[[0.15706636011600494, -0.6239964365959167, 0.8863904476165771, -0.7619400024414062, -1.2175467014312744, 0.3716956675052643, 0.29905179142951965, -0.4400036036968231, 0.3469213843345642, 0.7889828085899353, 0.4184873402118683, -0.9422792196273804, -1.2349071502685547, 0.83653324842453, 0.4640776216983795, -0.9603592157363892, -0.6297759413719177, -0.7400686144828796, -0.15474073588848114, -0.6766582727432251, -0.43687620759010315, -0.4734995663166046, -0.7973983883857727, -0.9379473924636841, 1.1111963987350464, -0.5142247676849365, 0.6504935026168823, -0.25692349672317505, 0.3403797447681427, -0.6943918466567993, 1.485856533050537, 1.3865001201629639, -0.8678648471832275, -1.1989864110946655, -0.8461583852767944, -0.160098597407341, -0.5765681266784668, 2.190514087677002, -0.1654336154460907, 0.109840027987957, -0.16620314121246338, 0.250108540058136, -1.5122675895690918, 0.47767558693885803, 1.3850675821304321, -0.43269261717796326, 0.15874409675598145, -0.8762330412864685, 0.554703

In [33]:
import json

def to_langchain_format(chapter):
    """
    Transforms a chapter dictionary into a standardized format expected by LangChain.

    Args:
        chapter (dict): A dictionary containing the keys 'chapter_title' and 'content'
                        which hold the title and content of the chapter, respectively.

    Returns:
        dict: A dictionary formatted for LangChain, with the title and cleaned content.
    """
    # Assuming 'content' is already cleaned
    return {
        "title": chapter["chapter_title"],
        "content": chapter["content"]  # already cleaned
    }



In [19]:
#data transformation for langchain format

import json

# Load the cleaned data
file_path = r'C:\Users\mharr\OneDrive\Documents\GitHub\MPEP_finetune\Notebooks\mpep_data_clean.json'
with open(file_path, 'r', encoding='utf-8') as file:
    cleaned_chapters = json.load(file)

# Apply the transformation to each chapter
processed_documents = [to_langchain_format(chapter) for chapter in cleaned_chapters]

# Optionally, save the processed documents to a new file
output_path = r'C:\Users\mharr\OneDrive\Documents\GitHub\MPEP_finetune\Notebooks\langchain_ready_data.json'
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(processed_documents, file, indent=4, ensure_ascii=False)

print("Processed data ready for LangChain saved to:", output_path)


Processed data ready for LangChain saved to: C:\Users\mharr\OneDrive\Documents\GitHub\MPEP_finetune\Notebooks\langchain_ready_data.json


In [27]:
#langchain documents object

from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir=r"C:\Users\mharr\OneDrive\Documents\GitHub\MPEP_finetune\Notebooks\langchainjson")
documents = reader.load_data()

In [29]:
#embeddings creation

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Load documents and build index
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 829/829 [00:39<00:00, 21.17it/s]


In [37]:
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [43]:
# save to disk

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=ollama_embedding
)

In [44]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=ollama_embedding,
)

In [58]:
# Query Data from the persisted index
query_engine = index.as_query_engine()
response = query_engine.query("What are the utility requirements of 35 USC 101? Please explain in detail.")
display(Markdown(f"<b>{response}</b>"))

<b>The utility requirements of 35 USC 101 necessitate that an invention must have a specific and substantial utility. This means that the invention must serve a useful purpose and provide a tangible benefit. The utility requirement ensures that the invention is not frivolous or devoid of any practical application. Additionally, the utility of the invention must be credible and specific, demonstrating that the invention is capable of achieving the disclosed utility. Overall, the utility requirement under 35 USC 101 aims to ensure that patented inventions contribute meaningfully to society and have a genuine purpose.</b>